In [ ]:
!gpu-who

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
import utils
from utils.import_for_notebooks import *

In [ ]:
from IPython.display import HTML, display
from utils.main_utils import html_vid
import vgpnn

vid_name = 'airballoons_QGAMTlI6XxY'

# Define GPNN
VGPNN, orig_vid = vgpnn.get_vgpnn(
    frames_dir=f'data/{vid_name}',
    start_frame=61,
    end_frame=75,
    device=device,
    max_size=100,
    min_size=(3, 15),  # (T,H,W)
    downfactor=(0.85, 0.85),
    J=5,
    J_start_from=2,
    kernel_size=(3,7,7),
    sthw=(1,1,1),
    reduce='median',
    vgpnn_type='pm',
)

print('Pyramid:')
for i, outs in enumerate(VGPNN.r_out_shapes):
    assert outs[0]>=VGPNN.Ks[i][0], 'kernel size must be bigger than size of video'
    print(f'LEVEL {i:3} SIZE {str(outs):17} KERNEL {str(VGPNN.Ks[i]):15} J {VGPNN.Js[i]}')

In [ ]:
z = torch.randn_like(VGPNN.q0).to(device)
save_dir = './results/airballoons/'
vgpnn_out = VGPNN.forward(
    VGPNN.q0, noises_dict={0: z}, noises_amps={0: 5}, verbose=False, save_dir=save_dir,
)

In [ ]:
anim_out = html_vid(vgpnn_out, interval=50)
anim_org = html_vid(orig_vid, interval=50)

display(HTML(f"""<table><tr>
<td>{anim_org.to_html5_video()}</td>
<td>{anim_out.to_html5_video()}</td>
</tr></table>"""))